In [22]:
import os
from openbabel import pybel
import Bio.PDB as bp

In [23]:
idk = os.walk("../data/raw_data/LIT-PCBA/AVE_unbiased")
walktup = next(idk)
walktup = next(idk)
prots = [ele for ele in walktup[2] if ele.endswith("protein.mol2")]
pdbfn = os.path.join(walktup[0], prots[0])
parser = bp.PDBParser()

In [30]:
import rdkit.Chem as Chem

In [32]:
len(Chem.SDMolSupplier("../data/raw_data/cyp_screen/test_3D_opt_1216.sdf"))

1216

In [40]:
len(Chem.SDMolSupplier("../data/testing_clean_regularly/output.sdf"))

1213

In [24]:
test = pybel.readfile("mol2", pdbfn)

In [25]:
mol = next(test)

*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL2 file (title is H_5o1i)



In [26]:
mol.removeh()

In [28]:
mol.write("pdb", "../data/testing_clean_regularly/test2.pdb", overwrite=True)

In [64]:
from openbabel import openbabel

In [66]:
converter = openbabel.OBConversion()

In [67]:
converter.SetInAndOutFormats("mol2", "pdb")

True

In [68]:
mol = openbabel.OBMol()

In [69]:
converter.ReadFile(mol, pdbfn)

*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL2 file (title is H_5o1i)



True

In [73]:
converter.WriteFile(mol, "../data/testing_clean_regularly/test.pdb")

True

In [72]:
os.getcwd()

'/home/qzj517/POR-DD/exploratory_notebooks'

In [60]:
import pybel

In [50]:
pdbfn = os.path.join(walktup[0], prots[0])

In [53]:
prot = Chem.MolFromMol2File(pdbfn)

In [58]:
print(Chem.MolToPDBBlock(prot))

COMPND    H_5o1i
HETATM    1  N1  UNL     1     115.314  73.879 -35.496  1.00  0.00           N1-
HETATM    2  C1  UNL     1     114.503  74.862 -36.216  1.00  0.00           C  
HETATM    3  C2  UNL     1     113.541  75.610 -35.298  1.00  0.00           C  
HETATM    4  O1  UNL     1     112.871  75.005 -34.449  1.00  0.00           O  
HETATM    5  C3  UNL     1     113.722  74.193 -37.344  1.00  0.00           C  
HETATM    6  O2  UNL     1     113.019  75.157 -38.110  1.00  0.00           O  
HETATM    7  N2  UNL     1     113.479  76.927 -35.480  1.00  0.00           N  
HETATM    8  C4  UNL     1     112.662  77.813 -34.652  1.00  0.00           C  
HETATM    9  C5  UNL     1     112.001  78.878 -35.522  1.00  0.00           C  
HETATM   10  O3  UNL     1     112.691  79.599 -36.256  1.00  0.00           O  
HETATM   11  C6  UNL     1     113.495  78.477 -33.543  1.00  0.00           C  
HETATM   12  C7  UNL     1     112.614  79.378 -32.707  1.00  0.00           C  
HETATM   13

In [56]:
prot

1511

In [3]:
pybel.readfile("mol2", pdbfn)

AttributeError: module 'pybel' has no attribute 'readfile'

In [55]:
mol2.PandasMol2().read_mol2(pdbfn)

ValueError: 9 columns passed, passed data had 10 columns